In [48]:
data = requests.get("https://api.socialverseapp.com/posts/view?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if").text

In [62]:
print(data)

{"status":"success","message":"Viewed posts fetched successfully","page":1,"page_size":1000,"max_page_size":1000,"posts":[{"id":1292,"category":{"id":22,"name":"SolTok","count":50,"description":"Ride the wave of Solana with SolTok","image_url":"https:\/\/assets.socialverseapp.com\/soltok.png"},"slug":"d4388c21bb5610d12a7f59f40b88d13473c5fb74","title":"Day 22 turning $100 into 100000 by trading meme coins #trading #memecoin #crypto #solana","identifier":"7LUUxsH","comment_count":3,"upvote_count":3,"view_count":68,"exit_count":497,"rating_count":13,"average_rating":26,"share_count":3,"video_link":"https:\/\/video-cdn.socialverseapp.com\/kinha_3948857c-b495-4d20-87ff-11d12643e589.mp4","contract_address":"7rhubtbunhohloxqs1powsr7hs4jkjvbuwgb3erzz6hh","chain_id":"solana","chart_url":"https:\/\/dexscreener.com\/solana\/7rhubtbunhohloxqs1powsr7hs4jkjvbuwgb3erzz6hh","baseToken":{"address":"BishJW8AN57nNUES2sDMNTNm8Nwic2CGPR5mXMwc1Vt5","name":"DevBased","symbol":"BASED","image_url":"https:\/\/d

In [82]:
import requests
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from datetime import datetime, timedelta
class SocialVideoRecommender:
    def __init__(self, flic_token):
        self.headers = {'Flic-Token': flic_token}
        self.base_url = 'https://api.socialverseapp.com'
        
    def _fetch_paginated_data(self, endpoint, params=None):
       
        if params is None:
            params = {}
            
        params['resonance_algorithm'] = 'resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if'
        
        all_data = []
        page = 1
        
        while True:
            try:
                current_params = {
                    **params,
                    'page': page,
                    'page_size': 1000
                }
                
                url = f"{self.base_url}{endpoint}"
                print(f"Fetching from {url}")
                
                response = requests.get(url, headers=self.headers, params=current_params)
                response.raise_for_status()
                
                data = response.json()
                
                # Handle the actual response structure
                if data.get('status') != 'success':
                    print(f"API returned non-success status: {data.get('status')}")
                    break
                
                # Extract posts array from response
                posts = data.get('posts', [])
                if not posts:
                    break
                    
                all_data.extend(posts)
                print(f"Received {len(posts)} items from page {page}")
                
                if len(posts) < current_params['page_size']:
                    break
                    
                page += 1
                
            except requests.exceptions.RequestException as e:
                print(f"Error fetching page {page} from {endpoint}: {str(e)}")
                break
                
        return {'data': all_data}
    def fetch_data(self):

        try:
        # Fetch all required data using correct endpoints
            self.posts = self._fetch_paginated_data('/posts/summary/get')
            self.views = self._fetch_paginated_data('/posts/view')
            self.likes = self._fetch_paginated_data('/posts/like')
            self.ratings = self._fetch_paginated_data('/posts/rating')
            self.users = self._fetch_paginated_data('/users/get_all')
        
        # Print fetched data statistics
            print(f"Fetched {len(self.posts.get('data', [])) if isinstance(self.posts, dict) else 0} posts")
            print(f"Fetched {len(self.views.get('data', [])) if isinstance(self.views, dict) else 0} views")
            print(f"Fetched {len(self.likes.get('data', [])) if isinstance(self.likes, dict) else 0} likes")
            print(f"Fetched {len(self.ratings.get('data', [])) if isinstance(self.ratings, dict) else 0} ratings")
            print(f"Fetched {len(self.users.get('data', [])) if isinstance(self.users, dict) else 0} users")
        
        except Exception as e:
            print(f"Error fetching data: {str(e)}")
            raise

    def preprocess_data(self):
    
        try:
            print("Starting data preprocessing...")
            
            # Initialize DataFrames for posts and interactions
            posts_data = pd.DataFrame(self.posts.get('data', []))
            
            print(f"Processing {len(posts_data)} posts...")

            # Create user-post interaction matrix
            interactions = defaultdict(lambda: defaultdict(float))
            
            # Process views (using view_count from posts)
            if not posts_data.empty:
                for _, post in posts_data.iterrows():
                    user_id = post.get('username')  # Using username as user_id
                    post_id = post.get('id')
                    if user_id and post_id:
                        view_weight = post.get('view_count', 0) * 1  # Weight for views
                        upvote_weight = post.get('upvote_count', 0) * 3  # Weight for upvotes
                        rating_weight = post.get('average_rating', 0) * post.get('rating_count', 0) * 5  # Weight for ratings
                        
                        total_interaction = view_weight + upvote_weight + rating_weight
                        interactions[user_id][post_id] = total_interaction
            
            print("Creating interaction matrix...")
            self.interaction_matrix = pd.DataFrame.from_dict(interactions, orient='index')
            self.interaction_matrix = self.interaction_matrix.fillna(0)
            
            print("Calculating content features...")
            if not posts_data.empty:
                features = []
                for _, post in posts_data.iterrows():
                    post_features = {
                        'id': post.get('id'),
                        'category_id': post.get('category', {}).get('id', 0),
                        'comment_count': post.get('comment_count', 0),
                        'upvote_count': post.get('upvote_count', 0),
                        'view_count': post.get('view_count', 0),
                        'rating_count': post.get('rating_count', 0),
                        'average_rating': post.get('average_rating', 0),
                        'share_count': post.get('share_count', 0),
                        'is_locked': 1 if post.get('is_locked') else 0,
                        'timestamp': post.get('created_at', 0)
                    }
                    features.append(post_features)
                    
                self.content_features = pd.DataFrame(features)
                if not self.content_features.empty:
                    self.content_features.set_index('id', inplace=True)
                    # Normalize features
                    numeric_columns = self.content_features.select_dtypes(include=[np.number]).columns
                    self.content_features[numeric_columns] = (
                        self.content_features[numeric_columns] - self.content_features[numeric_columns].mean()
                    ) / self.content_features[numeric_columns].std()
                    self.content_features = self.content_features.fillna(0)
            
            print("Calculating popularity scores...")
            self.popularity_scores = self._calculate_popularity_scores()
            
            print(f"Preprocessing complete:")
            print(f"- Processed {len(self.interaction_matrix)} user interactions")
            if hasattr(self, 'content_features'):
                print(f"- Generated features for {len(self.content_features)} posts")
                
        except Exception as e:
            print(f"Error preprocessing data: {str(e)}")
            traceback.print_exc()
            raise

    def get_recommendations(self, user_id, n_recommendations=10):
        """Get personalized recommendations for a user"""
        recommendations = []
        
        try:
            if not hasattr(self, 'interaction_matrix') or self.interaction_matrix.empty:
                # If no interaction data, return recommendations based on popularity
                recommendations = self._get_popular_recommendations(n_recommendations)
            elif user_id not in self.interaction_matrix.index:
                # For new users, return popularity-based recommendations
                recommendations = self._get_popular_recommendations(n_recommendations)
            else:
                # Get personalized recommendations for existing users
                recommendations = self._get_personalized_recommendations(user_id, n_recommendations)
            
        except Exception as e:
            print(f"Error getting recommendations: {str(e)}")
            # Fallback to simple popularity-based recommendations
            recommendations = self._get_popular_recommendations(n_recommendations)
        
        return recommendations
    
    def _get_popular_recommendations(self, n_recommendations):
        """Get recommendations based on popularity scores"""
        # Sort posts by popularity score
        sorted_posts = sorted(self.popularity_scores.items(), 
                            key=lambda x: x[1], 
                            reverse=True)
        
        return sorted_posts[:n_recommendations]
    
    def _get_personalized_recommendations(self, user_id, n_recommendations):
        """Get personalized recommendations based on user history"""
        # Get user's interaction history
        user_interactions = self.interaction_matrix.loc[user_id]
        
        # Get posts the user hasn't interacted with
        uninteracted_posts = [post_id for post_id in self.popularity_scores.keys()
                            if post_id not in user_interactions or user_interactions[post_id] == 0]
        
        # Calculate recommendation scores
        recommendations = []
        for post_id in uninteracted_posts:
            score = self.popularity_scores[post_id]  # Base score from popularity
            
            # Add content-based similarity if we have content features
            if hasattr(self, 'content_features') and not self.content_features.empty:
                if post_id in self.content_features.index:
                    similar_posts = self._get_similar_posts(post_id)
                    for similar_post_id, similarity in similar_posts:
                        if similar_post_id in user_interactions:
                            score += similarity * user_interactions[similar_post_id]
            
            recommendations.append((post_id, score))
        
        # Sort and return top N recommendations
        return sorted(recommendations, key=lambda x: x[1], reverse=True)[:n_recommendations]
    
    def _get_similar_posts(self, post_id):
        """Get similar posts based on content features"""
        if post_id not in self.content_features.index:
            return []
        
        # Calculate similarity with other posts
        post_features = self.content_features.loc[post_id].values.reshape(1, -1)
        similarities = cosine_similarity(post_features, self.content_features.values)[0]
        
        # Create list of (post_id, similarity) pairs
        similar_posts = list(zip(self.content_features.index, similarities))
        
        # Sort by similarity and remove the post itself
        similar_posts = sorted([(pid, sim) for pid, sim in similar_posts if pid != post_id], 
                             key=lambda x: x[1], 
                             reverse=True)

    def _calculate_popularity_scores(self):
        popularity = defaultdict(float)
        
        if isinstance(self.posts, dict) and 'data' in self.posts:
            current_time = datetime.now().timestamp()
            RECENCY_WINDOW = 7 * 24 * 60 * 60  # 7 days in seconds
            
            for post in self.posts['data']:
                post_id = post.get('id')
                if not post_id:
                    continue
                
                # Get post timestamp
                created_at = post.get('created_at', current_time)
                if isinstance(created_at, str):
                    try:
                        created_at = pd.to_datetime(created_at).timestamp()
                    except:
                        created_at = current_time
                
                # Calculate time decay factor
                time_diff = current_time - created_at
                recency_factor = max(0, 1 - (time_diff / RECENCY_WINDOW))
                
                # Calculate engagement score
                engagement = (
                    post.get('view_count', 0) * 1 +  # Views
                    post.get('upvote_count', 0) * 3 +  # Upvotes
                    post.get('rating_count', 0) * post.get('average_rating', 0) * 0.05 +  # Ratings
                    post.get('share_count', 0) * 4  # Shares
                )
                
                popularity[post_id] = engagement * recency_factor
            
            # Normalize scores
            if popularity:
                max_score = max(popularity.values())
                if max_score > 0:
                    popularity = {k: v/max_score for k, v in popularity.items()}
        
        return popularity
if __name__ == "__main__":
    try:
       
        recommender = SocialVideoRecommender(flic_token="flic_1e01009f9c1a54706f385bcc1993a08fd9647ba8f499572d280654d1c03c47bf")
        print("Fetching data...")
        recommender.fetch_data()
        print("\nPreprocessing data...")
        recommender.preprocess_data()
        user_id = "kinha"  # Example username from the response
        print(f"\nGetting recommendations for user {user_id}...")
        recommendations = recommender.get_recommendations(user_id)
        print(f"Top recommendations: {recommendations}")
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        print(traceback.format_exc())

Fetching data...
Fetching from https://api.socialverseapp.com/posts/summary/get
Received 1000 items from page 1
Fetching from https://api.socialverseapp.com/posts/summary/get
Received 3 items from page 2
Fetching from https://api.socialverseapp.com/posts/view
Received 1000 items from page 1
Fetching from https://api.socialverseapp.com/posts/view
Received 1000 items from page 2
Fetching from https://api.socialverseapp.com/posts/view
Received 1000 items from page 3
Fetching from https://api.socialverseapp.com/posts/view
Received 1000 items from page 4
Fetching from https://api.socialverseapp.com/posts/view
Received 216 items from page 5
Fetching from https://api.socialverseapp.com/posts/like
Received 1000 items from page 1
Fetching from https://api.socialverseapp.com/posts/like
Received 155 items from page 2
Fetching from https://api.socialverseapp.com/posts/rating
Received 1000 items from page 1
Fetching from https://api.socialverseapp.com/posts/rating
Received 1000 items from page 2
Fe

In [103]:
import requests
import os
import pandas as pd
import numpy as np
import traceback
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from datetime import datetime, timedelta

class SocialVideoRecommender:
    def __init__(self, flic_token):
        self.headers = {'Flic-Token': flic_token}
        self.base_url = 'https://api.socialverseapp.com'
        
    def _fetch_paginated_data(self, endpoint, params=None):
       
        if params is None:
            params = {}
            
        params['resonance_algorithm'] = 'resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if'
        
        all_data = []
        page = 1
        
        while True:
            try:
                current_params = {
                    **params,
                    'page': page,
                    'page_size': 1000
                }
                
                url = f"{self.base_url}{endpoint}"
                print(f"Fetching from {url}")
                
                response = requests.get(url, headers=self.headers, params=current_params)
                response.raise_for_status()
                
                data = response.json()
                
                # Handle the actual response structure
                if data.get('status') != 'success':
                    print(f"API returned non-success status: {data.get('status')}")
                    break
                
                # Extract posts array from response
                posts = data.get('posts', [])
                if not posts:
                    break
                    
                all_data.extend(posts)
                print(f"Received {len(posts)} items from page {page}")
                
                if len(posts) < current_params['page_size']:
                    break
                    
                page += 1
                
            except requests.exceptions.RequestException as e:
                print(f"Error fetching page {page} from {endpoint}: {str(e)}")
                break
                
        return {'data': all_data}
    def fetch_data(self):

        try:
        # Fetch all required data using correct endpoints
            self.posts = self._fetch_paginated_data('/posts/summary/get')
            self.views = self._fetch_paginated_data('/posts/view')
            self.likes = self._fetch_paginated_data('/posts/like')
            self.ratings = self._fetch_paginated_data('/posts/rating')
            self.users = self._fetch_paginated_data('/users/get_all')
        
        # Print fetched data statistics
            print(f"Fetched {len(self.posts.get('data', [])) if isinstance(self.posts, dict) else 0} posts")
            print(f"Fetched {len(self.views.get('data', [])) if isinstance(self.views, dict) else 0} views")
            print(f"Fetched {len(self.likes.get('data', [])) if isinstance(self.likes, dict) else 0} likes")
            print(f"Fetched {len(self.ratings.get('data', [])) if isinstance(self.ratings, dict) else 0} ratings")
            print(f"Fetched {len(self.users.get('data', [])) if isinstance(self.users, dict) else 0} users")
        
        except Exception as e:
            print(f"Error fetching data: {str(e)}")
            raise

    def preprocess_data(self):
    
        try:
            print("Starting data preprocessing...")
            
            # Initialize DataFrames for posts and interactions
            posts_data = pd.DataFrame(self.posts.get('data', []))
            
            print(f"Processing {len(posts_data)} posts...")

            # Create user-post interaction matrix
            interactions = defaultdict(lambda: defaultdict(float))
            
            # Process views (using view_count from posts)
            if not posts_data.empty:
                for _, post in posts_data.iterrows():
                    user_id = post.get('username')  # Using username as user_id
                    post_id = post.get('id')
                    if user_id and post_id:
                        view_weight = post.get('view_count', 0) * 1  # Weight for views
                        upvote_weight = post.get('upvote_count', 0) * 3  # Weight for upvotes
                        rating_weight = post.get('average_rating', 0) * post.get('rating_count', 0) * 5  # Weight for ratings
                        
                        total_interaction = view_weight + upvote_weight + rating_weight
                        interactions[user_id][post_id] = total_interaction
            
            print("Creating interaction matrix...")
            self.interaction_matrix = pd.DataFrame.from_dict(interactions, orient='index')
            self.interaction_matrix = self.interaction_matrix.fillna(0)
            
            print("Calculating content features...")
            if not posts_data.empty:
                features = []
                for _, post in posts_data.iterrows():
                    post_features = {
                        'id': post.get('id'),
                        'category_id': post.get('category', {}).get('id', 0),
                        'comment_count': post.get('comment_count', 0),
                        'upvote_count': post.get('upvote_count', 0),
                        'view_count': post.get('view_count', 0),
                        'rating_count': post.get('rating_count', 0),
                        'average_rating': post.get('average_rating', 0),
                        'share_count': post.get('share_count', 0),
                        'is_locked': 1 if post.get('is_locked') else 0,
                        'timestamp': post.get('created_at', 0)
                    }
                    features.append(post_features)
                    
                self.content_features = pd.DataFrame(features)
                if not self.content_features.empty:
                    self.content_features.set_index('id', inplace=True)
                    # Normalize features
                    numeric_columns = self.content_features.select_dtypes(include=[np.number]).columns
                    self.content_features[numeric_columns] = (
                        self.content_features[numeric_columns] - self.content_features[numeric_columns].mean()
                    ) / self.content_features[numeric_columns].std()
                    self.content_features = self.content_features.fillna(0)
            
            print("Calculating popularity scores...")
            self.popularity_scores = self._calculate_popularity_scores()
            
            print(f"Preprocessing complete:")
            print(f"- Processed {len(self.interaction_matrix)} user interactions")
            if hasattr(self, 'content_features'):
                print(f"- Generated features for {len(self.content_features)} posts")
                
        except Exception as e:
            print(f"Error preprocessing data: {str(e)}")
            traceback.print_exc()
            raise

    def get_recommendations(self, user_id, n_recommendations=10):
        """Get personalized recommendations for a user"""
        recommendations = []
        
        try:
            if not hasattr(self, 'interaction_matrix') or self.interaction_matrix.empty:
                # If no interaction data, return recommendations based on popularity
                recommendations = self._get_popular_recommendations(n_recommendations)
            elif user_id not in self.interaction_matrix.index:
                # For new users, return popularity-based recommendations
                recommendations = self._get_popular_recommendations(n_recommendations)
            else:
                # Get personalized recommendations for existing users
                recommendations = self._get_personalized_recommendations(user_id, n_recommendations)
            
        except Exception as e:
            print(f"Error getting recommendations: {str(e)}")
            # Fallback to simple popularity-based recommendations
            recommendations = self._get_popular_recommendations(n_recommendations)
        
        return recommendations
    
    def _get_popular_recommendations(self, n_recommendations):
        """Get recommendations based on popularity scores"""
        # Sort posts by popularity score
        sorted_posts = sorted(self.popularity_scores.items(), 
                            key=lambda x: x[1], 
                            reverse=True)
        
        return sorted_posts[:n_recommendations]
    
    def _get_personalized_recommendations(self, user_id, n_recommendations):
        """Get personalized recommendations based on user history"""
        # Get user's interaction history
        user_interactions = self.interaction_matrix.loc[user_id]
        
        # Get posts the user hasn't interacted with
        uninteracted_posts = [post_id for post_id in self.popularity_scores.keys()
                            if post_id not in user_interactions or user_interactions[post_id] == 0]
        
        # Calculate recommendation scores
        recommendations = []
        for post_id in uninteracted_posts:
            score = self.popularity_scores[post_id]  # Base score from popularity
            
            # Add content-based similarity if we have content features
            if hasattr(self, 'content_features') and not self.content_features.empty:
                if post_id in self.content_features.index:
                    similar_posts = self._get_similar_posts(post_id)
                    for similar_post_id, similarity in similar_posts:
                        if similar_post_id in user_interactions:
                            score += similarity * user_interactions[similar_post_id]
            
            recommendations.append((post_id, score))
        
        # Sort and return top N recommendations
        return sorted(recommendations, key=lambda x: x[1], reverse=True)[:n_recommendations]
    
    def _get_similar_posts(self, post_id):
        """Get similar posts based on content features"""
        if post_id not in self.content_features.index:
            return []
        
        # Calculate similarity with other posts
        post_features = self.content_features.loc[post_id].values.reshape(1, -1)
        similarities = cosine_similarity(post_features, self.content_features.values)[0]
        
        # Create list of (post_id, similarity) pairs
        similar_posts = list(zip(self.content_features.index, similarities))
        
        # Sort by similarity and remove the post itself
        similar_posts = sorted([(pid, sim) for pid, sim in similar_posts if pid != post_id], 
                             key=lambda x: x[1], 
                             reverse=True)

    def _calculate_popularity_scores(self):
        popularity = defaultdict(float)
        
        if isinstance(self.posts, dict) and 'data' in self.posts:
            current_time = datetime.now().timestamp()
            RECENCY_WINDOW = 7 * 24 * 60 * 60  # 7 days in seconds
            
            for post in self.posts['data']:
                post_id = post.get('id')
                if not post_id:
                    continue
                
                # Get post timestamp
                created_at = post.get('created_at', current_time)
                if isinstance(created_at, str):
                    try:
                        created_at = pd.to_datetime(created_at).timestamp()
                    except:
                        created_at = current_time
                
                # Calculate time decay factor
                time_diff = current_time - created_at
                recency_factor = max(0, 1 - (time_diff / RECENCY_WINDOW))
                
                # Calculate engagement score
                engagement = (
                    post.get('view_count', 0) * 1 +  # Views
                    post.get('upvote_count', 0) * 3 +  # Upvotes
                    post.get('rating_count', 0) * post.get('average_rating', 0) * 0.05 +  # Ratings
                    post.get('share_count', 0) * 4  # Shares
                )
                
                popularity[post_id] = engagement * recency_factor
            
            # Normalize scores
            if popularity:
                max_score = max(popularity.values())
                if max_score > 0:
                    popularity = {k: v/max_score for k, v in popularity.items()}
        
        return popularity

    def get_recommendations(self, user_id, n_recommendations=10):
        
        recommendations = []
        
        try:
            if not hasattr(self, 'interaction_matrix') or self.interaction_matrix.empty:
                # If no interaction data, return recommendations based on popularity
                recommendations = self._get_popular_recommendations(n_recommendations)
            elif user_id not in self.interaction_matrix.index:
                # For new users, return popularity-based recommendations
                recommendations = self._get_popular_recommendations(n_recommendations)
            else:
                # Get personalized recommendations for existing users
                recommendations = self._get_personalized_recommendations(user_id, n_recommendations)
            
        except Exception as e:
            print(f"Error getting recommendations: {str(e)}")
            # Fallback to simple popularity-based recommendations
            recommendations = self._get_popular_recommendations(n_recommendations)
        
        return recommendations

    def _enrich_recommendations(self, recommendations):
        enriched_recommendations = []
        
        # Create a mapping of post IDs to full post details
        post_map = {post.get('id'): post for post in self.posts.get('data', [])}
        
        for post_id, score in recommendations:
            post_details = post_map.get(post_id, {})
            enriched_recommendation = {
                'id': post_id,
                'name': post_details.get('title', 'Unknown Title'),
                'link': post_details.get('link', ''),
                'score': score
            }
            enriched_recommendations.append(enriched_recommendation)
        
        return enriched_recommendations
    def save_recommendations_to_csv(self, user_id, recommendations, filename=None):
        try:
        # Enrich recommendations with post details
            enriched_recommendations = self._enrich_recommendations(recommendations)
        
        # Create DataFrame from enriched recommendations
            recommendations_df = pd.DataFrame(enriched_recommendations)
        
        # Add user_id column
            recommendations_df['user_id'] = user_id
        
        # Add timestamp column
            recommendations_df['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Generate default filename if not provided
            if filename is None:
                filename = f"recommendations_{user_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        
        # Ensure recommendations directory exists
            os.makedirs('recommendations', exist_ok=True)
        
        # Full path for saving
            filepath = os.path.join('recommendations', filename)
        
        # Save to CSV
            recommendations_df.to_csv(filepath, index=False)
        
            print(f"Recommendations saved to {filepath}")
            return filepath
    
        except Exception as e:
            print(f"Error saving recommendations to CSV: {str(e)}")
            traceback.print_exc()
            return None

if __name__ == "__main__":
    try:
        recommender = SocialVideoRecommender(flic_token="flic_1e01009f9c1a54706f385bcc1993a08fd9647ba8f499572d280654d1c03c47bf")
        print("Fetching data...")
        recommender.fetch_data()
        print("\nPreprocessing data...")
        recommender.preprocess_data()
        
        user_id = "kinha"  # Example username from the response
        print(f"\nGetting recommendations for user {user_id}...")
        recommendations = recommender.get_recommendations(user_id)
        
        # Enrich recommendations with post details
        enriched_recommendations = recommender._enrich_recommendations(recommendations[:4])
        csv_path = recommender.save_recommendations_to_csv(user_id, recommendations)
        
        # Optional: Print recommendations
        if csv_path:
            print("\nTop Recommendations:")
            recommendations_df = pd.read_csv(csv_path)
            print(recommendations_df)
        
        print("Top Recommendations:")
        for idx, rec in enumerate(enriched_recommendations, 1):
            print(f"{idx}. Name: {rec['name']}")
            print(f"   ID: {rec['id']}")
            print(f"   Link: {rec['link']}")
            print(f"   Score: {rec['score']:.4f}\n")
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print(traceback.format_exc())

Fetching data...
Fetching from https://api.socialverseapp.com/posts/summary/get
Received 1000 items from page 1
Fetching from https://api.socialverseapp.com/posts/summary/get
Received 3 items from page 2
Fetching from https://api.socialverseapp.com/posts/view
Received 1000 items from page 1
Fetching from https://api.socialverseapp.com/posts/view
Received 1000 items from page 2
Fetching from https://api.socialverseapp.com/posts/view
Received 1000 items from page 3
Fetching from https://api.socialverseapp.com/posts/view
Received 1000 items from page 4
Fetching from https://api.socialverseapp.com/posts/view
Received 216 items from page 5
Fetching from https://api.socialverseapp.com/posts/like
Received 1000 items from page 1
Fetching from https://api.socialverseapp.com/posts/like
Received 155 items from page 2
Fetching from https://api.socialverseapp.com/posts/rating
Received 1000 items from page 1
Fetching from https://api.socialverseapp.com/posts/rating
Received 1000 items from page 2
Fe